# Calibration and Real World Error Calculation for 20 top-view images

###### Calibration has been done including all the images but real world error is calculated only across the 20 top-view images

In [13]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [14]:
dirs = ["./180/"]

In [ ]:
if not (os.path.exists("./vid_hard_check/")):
    os.makedirs("./vid_hard_check/")

In [16]:
global_dict_no_eq = {}
for d in dirs:
    flist = sorted(glob.glob(os.path.join(d, "*.mp4")))
    for vf in flist:
        print(vf)
        vidcap = cv2.VideoCapture(vf)

        files = glob.glob("./vid_hard_check/*")
        for fl in files:
            os.remove(fl)

        success,image = vidcap.read()
        count = 0
        success = True
        while success:
            success,image = vidcap.read()
            cv2.imwrite("./vid_hard_check/frame%d.png" % count, image)     # save frame as PNG file
            if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                break
            count += 1

        print(count)

        nx = 5
        ny = 5
        bins = 6
        
        
        interval = count/bins

        mesh = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
        objp = np.hstack((mesh, np.zeros(shape = (nx*ny,1))))
        objp = np.array(objp, dtype = np.float32)*20.0

        fname = glob.glob('./vid_hard_check/*.png')
        
        for j in range(1,4):

            imgpoints = []
            imeval = []
            objpoints = []
            imgpoints2_list = []
            detected_list = []
            bin_error = []
            im_counter = 0
            bincount = np.zeros(6)
            fx = j*0.1
            fy = j*0.1
            for i in range(len(fname)):
            #     for i in range(1,11):
                try:
                    f = "./vid_hard_check/frame{}.png".format(i+1)
                    imcv = cv2.imread(f)
                    imcv = cv2.cvtColor(imcv, cv2.COLOR_BGR2GRAY)

####### Uncomment below two lines to apply histogram equalization for removing shiny effects, cl = 2, 5 or 10 work best #####                    

#                     cli = cv2.createCLAHE(clipLimit = cl)
#                     imcv = cli.apply(imcv)

                    imcv = cv2.GaussianBlur(imcv, ksize = (5,5), sigmaX = 1, sigmaY = 1)
                    imcv = cv2.resize(imcv, (0, 0), fx=fx, fy=fy, interpolation=cv2.INTER_AREA)
                    img = imcv
                    retval, corners = cv2.findChessboardCorners(imcv,(nx,ny), None)
                    if (retval == True):
                        detected_list.append(f)
                        if ((i < int(count/2 + 10)) and (i >= int(count/2 - 10))):
                            imeval.append((corners,im_counter))
                    
                        imgpoints.append(corners)
                        objpoints.append(objp)
                        im_counter += 1
                except Exception as e:
                    print("Exception occurred: {}".format(e))

            print(fx, len(imeval))
            mean_error = 1.0
            mtx = dist = rvecs = tvecs = []
            if (len(objpoints) != 0):
                retval, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[::-1], None, None)
                mean_error = 0.0
                
                for i in range(len(imeval)):
                    sp = imeval[i][0]
                    objp = objpoints[i]
                    src = np.array([[[sp[0,0,0], sp[0,0,1]], [sp[4,0,0], sp[4,0,1]], [sp[24,0,0], sp[24,0,1]], [sp[20,0,0], sp[20,0,1]]]],dtype=np.float32)
                    dst = np.array([[[objp[0,0], objp[0,1]], [objp[4,0], objp[4,1]], [objp[24,0], objp[24,1]], [objp[20,0], objp[20,1]]]],dtype=np.float32)
                    PM = cv2.getPerspectiveTransform(src,dst)
                    ind = imeval[i][1]
                    imgpoints2, _ = cv2.projectPoints(objpoints[ind], rvecs[ind], tvecs[ind], mtx, dist)
                    sp = np.reshape(sp, newshape=(1,sp.shape[0], sp.shape[2]))
                    dp = np.reshape(imgpoints2, newshape=(1,imgpoints2.shape[0], 2))
                    slanded_real = cv2.perspectiveTransform(sp,PM)
                    dlanded_real = cv2.perspectiveTransform(dp,PM)
                    res = cv2.norm(slanded_real, dlanded_real, cv2.NORM_L2)
                    res = np.sqrt(res**2/slanded_real.shape[1])
                    mean_error += res
                print( "total error: {} for scaling of {} with {} number of images for file {}".format(mean_error/(len(imeval)+0.00001), fx, len(imeval), vf))

            if vf not in global_dict_no_eq:
                global_dict_no_eq[vf] = []

            global_dict_no_eq[vf].append([fx, len(imeval), mean_error/(len(imeval)+0.00001), mtx, dist])

./180/c1_180.mp4
512
Exception occurred
Exception occurred
0.1 20
total error: 0.2623064735592111 for scaling of 0.1 with 20 number of images for file ./180/c1_180.mp4
Exception occurred
Exception occurred
0.2 20
total error: 0.23685816354426892 for scaling of 0.2 with 20 number of images for file ./180/c1_180.mp4
Exception occurred
Exception occurred
0.30000000000000004 20
total error: 0.22526598765421452 for scaling of 0.30000000000000004 with 20 number of images for file ./180/c1_180.mp4
./180/c3_180.mp4
518
Exception occurred
Exception occurred
0.1 18
total error: 0.33512717147664084 for scaling of 0.1 with 18 number of images for file ./180/c3_180.mp4
Exception occurred
Exception occurred
0.2 20
total error: 0.564073005815647 for scaling of 0.2 with 20 number of images for file ./180/c3_180.mp4
Exception occurred
Exception occurred
0.30000000000000004 20
total error: 0.5757731270615846 for scaling of 0.30000000000000004 with 20 number of images for file ./180/c3_180.mp4
./180/c2_1

## Plotting the real-world errors

In [19]:
plotter = []

In [20]:
for key in sorted(flist):
    l = global_dict_no_eq[key]
    plot = []
    plot.append(l[0][2])
    plot.append(l[1][2])
    plot.append(l[2][2])
    plotter.append(plot)

In [21]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets

py.offline.init_notebook_mode(connected=True)

In [22]:
layout1 = go.Layout(
    title='Real RMSE Error Plot',
    yaxis = dict(title='error'),
    xaxis = dict(title='fx')
)

trace = []

for i in range(4):
    treace_temp = go.Scatter(
    x = (np.arange(3)+1)*0.1,
    y = plotter[i],
    mode='lines+markers'
    )
    trace.append(trace_temp)

In [23]:
fig = go.Figure(data=trace, layout=layout1)

In [24]:
py.offline.iplot(fig)